# Fetch Reddit Engagement Statistics

This notebook fetches and displays real-time Reddit engagement statistics for posted responses using the analytics service.

**Features:**
- Real-time karma and engagement metrics from Reddit
- Posting history and success rates
- Detailed analytics for each posted response
- Performance tracking over time

## Cell 1: Setup and Fetch Engagement Report

Initialize services and fetch the latest engagement report from Reddit.

In [ ]:
import asyncio
import json
import os
import sys
from datetime import datetime

# Add the parent directory to the system path to allow importing from src
sys.path.insert(0, os.path.abspath('../'))

from src.config.settings import settings
from src.storage.json_storage import JsonStorage
from src.clients.reddit_client import RedditClient
from src.services.analytics_service import AnalyticsService

# Initialize services
json_storage = JsonStorage()
reddit_client = RedditClient(
    client_id=settings.REDDIT_CLIENT_ID,
    client_secret=settings.REDDIT_CLIENT_SECRET,
    username=settings.REDDIT_USERNAME,
    password=settings.REDDIT_PASSWORD
)
analytics_service = AnalyticsService(json_storage, reddit_client)

# Load organization ID from previous workflow
print("Loading organization information...")
try:
    # Try to load from ingested docs first
    organization_id = None
    
    if os.path.exists('ingested_docs.json'):
        with open('ingested_docs.json', 'r') as f:
            ingested_data = json.load(f)
        organization_id = ingested_data.get('organization_id')
    
    # Fallback to other files if needed
    if not organization_id and os.path.exists('selected_subreddits.json'):
        with open('selected_subreddits.json', 'r') as f:
            subreddit_data = json.load(f)
        organization_id = subreddit_data.get('organization_id')
    
    if organization_id:
        print(f"🏢 Organization ID: {organization_id}")
    else:
        print(f"⚠️ No organization ID found, will analyze all posted responses")
        organization_id = None
    
    # Fetch engagement report
    print(f"\n📊 Fetching engagement report from Reddit...")
    print(f"⏳ This may take a moment as we fetch live data from Reddit API...")
    
    engagement_report = await analytics_service.get_engagement_report(
        organization_id=organization_id
    )
    
    print(f"\n✅ Engagement report fetched successfully!")
    
    # Display summary statistics
    print(f"\n📈 ENGAGEMENT SUMMARY")
    print(f"{'='*50}")
    print(f"Total Responses: {engagement_report.get('total_responses', 0)}")
    print(f"Successful Posts: {engagement_report.get('successful_responses', 0)}")
    print(f"Failed Posts: {engagement_report.get('failed_responses', 0)}")
    
    if engagement_report.get('successful_responses', 0) > 0:
        success_rate = (engagement_report.get('successful_responses', 0) / engagement_report.get('total_responses', 1)) * 100
        print(f"Success Rate: {success_rate:.1f}%")
        print(f"Total Karma Earned: {engagement_report.get('total_karma_earned', 0)}")
        print(f"Average Karma per Response: {engagement_report.get('average_karma_per_response', 0)}")
    
    # Display response type breakdown
    response_types = engagement_report.get('response_type_breakdown', {})
    if response_types:
        print(f"\n📝 RESPONSE TYPE BREAKDOWN")
        print(f"{'='*50}")
        for response_type, count in response_types.items():
            print(f"{response_type}: {count}")
    
    # Display detailed engagement metrics
    engagement_metrics = engagement_report.get('engagement_metrics', {})
    if engagement_metrics:
        print(f"\n🎯 DETAILED ENGAGEMENT METRICS")
        print(f"{'='*50}")
        
        for response_id, metrics in engagement_metrics.items():
            if 'error' in metrics:
                print(f"\n❌ Response {response_id[:8]}... (Error fetching data)")
                print(f"   Error: {metrics['error']}")
            else:
                print(f"\n📊 Response {response_id[:8]}... ({metrics.get('target_type', 'unknown')})")
                print(f"   Current Score: {metrics.get('current_score', 'N/A')}")
                print(f"   Response Type: {metrics.get('response_type', 'N/A')}")
                print(f"   Posted: {metrics.get('posted_at', 'N/A')}")
                
                if 'comment_count' in metrics:
                    print(f"   Comments: {metrics['comment_count']}")
    
    print(f"\n⏰ Report generated: {engagement_report.get('last_updated', 'Unknown')}")
    
    # Save engagement report
    with open('reddit_engagement_report.json', 'w') as f:
        json.dump(engagement_report, f, indent=2)
    
    print(f"\n📁 Saved engagement report to reddit_engagement_report.json")
    
    # Check if there are any posted responses
    if engagement_report.get('total_responses', 0) == 0:
        print(f"\n💡 No posted responses found. Run the discover_posts_respond.ipynb notebook first to post some responses!")
    elif engagement_report.get('successful_responses', 0) == 0:
        print(f"\n⚠️ No successful posts found. Check the posting results in discover_posts_respond.ipynb")
    else:
        print(f"\n🎉 Analytics complete! Your Reddit marketing campaign is active.")

except Exception as e:
    print(f"❌ Error fetching engagement report: {str(e)}")
    
    # Save error info
    error_report = {
        "error": str(e),
        "timestamp": datetime.now().isoformat(),
        "organization_id": organization_id
    }
    
    with open('reddit_engagement_report.json', 'w') as f:
        json.dump(error_report, f, indent=2)

## Cell 2: Fetch Posting History

Get detailed posting history and performance metrics.

In [ ]:
import asyncio
import json
import os
import sys

# Add the parent directory to the system path to allow importing from src
sys.path.insert(0, os.path.abspath('../'))

from src.config.settings import settings
from src.storage.json_storage import JsonStorage
from src.clients.reddit_client import RedditClient
from src.services.analytics_service import AnalyticsService

# Initialize services (reuse from previous cell if needed)
json_storage = JsonStorage()
reddit_client = RedditClient(
    client_id=settings.REDDIT_CLIENT_ID,
    client_secret=settings.REDDIT_CLIENT_SECRET,
    username=settings.REDDIT_USERNAME,
    password=settings.REDDIT_PASSWORD
)
analytics_service = AnalyticsService(json_storage, reddit_client)

# Load organization ID
organization_id = None
if os.path.exists('ingested_docs.json'):
    with open('ingested_docs.json', 'r') as f:
        ingested_data = json.load(f)
    organization_id = ingested_data.get('organization_id')

print(f"📚 Fetching posting history...")

try:
    # Get posting history
    posting_history = analytics_service.get_posting_history(
        organization_id=organization_id,
        limit=20  # Last 20 posts
    )
    
    print(f"\n📋 POSTING HISTORY")
    print(f"{'='*60}")
    print(f"Total Responses: {posting_history.get('total_responses', 0)}")
    print(f"Successful: {posting_history.get('successful_responses', 0)}")
    print(f"Failed: {posting_history.get('failed_responses', 0)}")
    print(f"Success Rate: {posting_history.get('success_rate', 0)}%")
    
    # Display recent responses
    recent_responses = posting_history.get('recent_responses', [])
    if recent_responses:
        print(f"\n🕒 RECENT RESPONSES (Last {len(recent_responses)})")
        print(f"{'='*60}")
        
        for i, response in enumerate(recent_responses, 1):
            status_icon = "✅" if response.get('success') else "❌"
            print(f"\n{status_icon} {i}. {response.get('response_type', 'unknown').upper()}")
            print(f"   Posted: {response.get('posted_at', 'Unknown')}")
            print(f"   Target ID: {response.get('target_id', 'N/A')}")
            
            if response.get('success'):
                print(f"   Status: Posted successfully")
            else:
                print(f"   Error: {response.get('error', 'Unknown error')}")
            
            content_preview = response.get('content_preview', '')
            if content_preview:
                print(f"   Preview: {content_preview}")
    else:
        print(f"\n💡 No posting history found. Run discover_posts_respond.ipynb to start posting!")
    
    # Save posting history
    with open('posting_history_report.json', 'w') as f:
        json.dump(posting_history, f, indent=2)
    
    print(f"\n📁 Saved posting history to posting_history_report.json")

except Exception as e:
    print(f"❌ Error fetching posting history: {str(e)}")

## Cell 3: Performance Analysis and Recommendations

Analyze performance data and provide recommendations for improvement.

In [ ]:
import json
import os
from datetime import datetime

print(f"🔍 PERFORMANCE ANALYSIS")
print(f"{'='*60}")

try:
    # Load engagement report
    if os.path.exists('reddit_engagement_report.json'):
        with open('reddit_engagement_report.json', 'r') as f:
            engagement_data = json.load(f)
    else:
        print("❌ No engagement report found. Run Cell 1 first.")
        engagement_data = {}
    
    # Load posting history
    if os.path.exists('posting_history_report.json'):
        with open('posting_history_report.json', 'r') as f:
            history_data = json.load(f)
    else:
        print("❌ No posting history found. Run Cell 2 first.")
        history_data = {}
    
    # Analyze performance
    total_responses = engagement_data.get('total_responses', 0)
    successful_responses = engagement_data.get('successful_responses', 0)
    total_karma = engagement_data.get('total_karma_earned', 0)
    avg_karma = engagement_data.get('average_karma_per_response', 0)
    
    print(f"\n📊 KEY METRICS")
    print(f"{'-'*30}")
    print(f"Total Posts: {total_responses}")
    print(f"Successful Posts: {successful_responses}")
    print(f"Total Karma: {total_karma}")
    print(f"Average Karma: {avg_karma:.2f}")
    
    # Performance recommendations
    print(f"\n💡 RECOMMENDATIONS")
    print(f"{'-'*30}")
    
    if total_responses == 0:
        print(f"🚀 Get started by running the discover_posts_respond.ipynb notebook")
        print(f"📝 Focus on high-quality, helpful responses to build engagement")
    elif successful_responses == 0:
        print(f"⚠️ No successful posts yet. Check your Reddit credentials and API limits")
        print(f"🔧 Review error messages in the posting history")
    else:
        success_rate = (successful_responses / total_responses) * 100
        
        if success_rate < 50:
            print(f"🔧 Low success rate ({success_rate:.1f}%). Check for API errors or rate limiting")
        elif success_rate < 80:
            print(f"📈 Moderate success rate ({success_rate:.1f}%). Room for improvement")
        else:
            print(f"✅ Excellent success rate ({success_rate:.1f}%)!")
        
        if avg_karma < 1:
            print(f"📝 Low average karma. Focus on more helpful and relevant responses")
            print(f"🎯 Target posts with more engagement potential")
        elif avg_karma < 5:
            print(f"📊 Moderate karma performance. Consider improving response quality")
        else:
            print(f"🏆 Great karma performance! Your responses are well-received")
        
        # Response type analysis
        response_types = engagement_data.get('response_type_breakdown', {})
        if response_types:
            print(f"\n📋 Response Type Performance:")
            for resp_type, count in response_types.items():
                print(f"   {resp_type}: {count} responses")
    
    # Best performing responses
    engagement_metrics = engagement_data.get('engagement_metrics', {})
    if engagement_metrics:
        # Find top performing responses
        top_responses = []
        for resp_id, metrics in engagement_metrics.items():
            if 'current_score' in metrics and isinstance(metrics['current_score'], (int, float)):
                top_responses.append((resp_id, metrics['current_score'], metrics))
        
        if top_responses:
            top_responses.sort(key=lambda x: x[1], reverse=True)
            
            print(f"\n🏆 TOP PERFORMING RESPONSES")
            print(f"{'-'*40}")
            
            for i, (resp_id, score, metrics) in enumerate(top_responses[:5], 1):
                print(f"{i}. Score: {score} | Type: {metrics.get('response_type', 'N/A')}")
                print(f"   Posted: {metrics.get('posted_at', 'N/A')}")
    
    # Generate summary report
    summary_report = {
        "analysis_timestamp": datetime.now().isoformat(),
        "performance_summary": {
            "total_responses": total_responses,
            "successful_responses": successful_responses,
            "success_rate": (successful_responses / total_responses * 100) if total_responses > 0 else 0,
            "total_karma_earned": total_karma,
            "average_karma_per_response": avg_karma
        },
        "recommendations": {
            "focus_areas": [],
            "next_steps": []
        }
    }
    
    # Add specific recommendations based on performance
    if total_responses == 0:
        summary_report["recommendations"]["next_steps"].append("Start posting responses using discover_posts_respond.ipynb")
    elif avg_karma < 2:
        summary_report["recommendations"]["focus_areas"].append("Improve response quality and relevance")
    
    with open('performance_analysis_report.json', 'w') as f:
        json.dump(summary_report, f, indent=2)
    
    print(f"\n📁 Saved performance analysis to performance_analysis_report.json")
    print(f"\n✅ Analysis complete! Use these insights to optimize your Reddit marketing strategy.")

except Exception as e:
    print(f"❌ Error during performance analysis: {str(e)}")